In [8]:
from hyppo.independence import Dcorr
from hyppo.ksample._utils import k_sample_transform
from mvlearn.embed import GCCA
import numpy as np

In [22]:
subjs = np.tile(np.arange(60), (3)).reshape((6,-1))

In [28]:
def simulate_data(subjs, n=1000, d=100):
    subj2vec = dict()
    for subj in np.unique(np.concatenate(subjs)):
        subj2vec[subj] = np.random.normal(0,1,(n,d))
    groups = []
    for subj_list in subjs:
        groups.append([np.random.normal(subj2vec[subj],0.1) for subj in subj_list])
    return groups

In [29]:
groups = simulate_data(subjs)

In [40]:
groups = np.asarray(groups).reshape((60*3, 1000, 100))

In [42]:
gcca = GCCA(n_components=3)
gcca = gcca.fit(groups)

In [43]:
gradients = gcca.transform(groups)

In [44]:
gradients.shape

(180, 1000, 3)

In [47]:
subjs.reshape((60*3)).reshape(6,30)

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
        46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
        46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
        46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]])

In [48]:
X, Y = k_sample_transform(
    gradients.reshape((6,30,1000,3))
)

In [55]:
for i in range(3):
    print(f'Gradient {i}')
    X0 = X[:,:,i].reshape(X.shape[0], -1)
    
    stat, pvalue = Dcorr().test(
        X0, Y,
        reps=1000,
        workers=-1,
        auto=False,
        perm_blocks=np.concatenate(subjs),
    )
    
    print(f'-pvalue: {pvalue}')

Gradient 0
-pvalue: 0.371
Gradient 1
-pvalue: 0.327
Gradient 2
-pvalue: 0.944
